In [1]:
#Lane detection for Autonoumous car - Simple algorithms by VINIL

In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/ slope)
    x2 = int((y2 - intercept)/ slope)
    return np.array([x1, y1, x2, y2])

In [6]:
def average_slope_intercept(image, lines):
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameter = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameter[0]
        intercept = parameter[1]
        if slope > 0 :
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])       

In [7]:
def canny (image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0 )
    canny = cv2.Canny(blur, 50 , 150)
    return canny

In [8]:
def display_lines(image , lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines: 
             cv2.line(line_image, (x1,y1), (x2,y2), (255,0,0), 10)
    return line_image

In [9]:
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(200, height) , (1100, height), (550,250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons , 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image    

In [10]:
image= cv2.imread('test_image.jpg')

In [11]:
#image

In [12]:
lane_image = np.copy(image)

In [13]:
canny_image=canny(lane_image)

In [14]:
#canny_image

In [15]:
cropped_image = region_of_interest(canny_image)

In [16]:
lines = cv2.HoughLinesP(cropped_image,2, np.pi/180,  100, np.array([]), minLineLength=40, maxLineGap=5)

In [17]:
averaged_lines = average_slope_intercept(lane_image, lines)

In [18]:
line_image = display_lines(lane_image, averaged_lines)

In [19]:
combo_image = cv2.addWeighted(lane_image, 0.8 , line_image, 1, 1)

In [20]:
#cv2.imshow("results",combo_image)

In [21]:
cv2.imshow("results",combo_image)

In [22]:
cv2.waitKey(0)

-1